In [ ]:
import os
from os import sys
sys.path.append('../../MpowerAI')
from pympower.classes.mshaai import MShaAI

shaai = MShaAI()

In [ ]:
srcPath = "./data/경조사지원규정.pdf"
tgtName = "경조사지원규정.txt"
OUT_FOLDER = "./data/out"

if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

tgtPath = f"{OUT_FOLDER}/{tgtName}"

shaai.extract(srcPath=srcPath, tgtPath=tgtPath)

In [ ]:
# URL 이미지 => 저장 하는 예제
import requests
from io import BytesIO
from PIL import Image

def download_save(url:str, max_size=(128, 128)):
    save_path = "./data/test.jpg"
    
    #이미지다운로드
    response = requests.get(url)
    if response.status_code == 200:
        # 이미지 열기
        img = Image.open(BytesIO(response.content))
      
        # [bong][2024-06-28] 
        # 노래만들기 할때 .PNG 파일 업로드 하면 에러남.
        # => PNG 파일은 투명도를 포함할 수 있는 RGBA 모드를 사용할 수 있는데, 그러나 JPEG는 투명도를 지원하지 않기 때문에 에러남.
        #=> 따라서 PNG 파일을 JPEG로 변환시 투명 부분을 흰색으로 처리함.
        if img.mode == 'RGBA':
            # 흰색 배경의 새로운 이미지 생성
            background = Image.new("RGB", img.size, (255, 255, 255))
            # 기존 이미지를 배경 이미지에 덮어쓰기
            background.paste(img, (0, 0), img)
            img = background
            
        # 이미지 크기 확인 및 조정
        if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
            img.thumbnail(max_size)
            
        # JPG 형식으로 저장
        img.save(save_path, "JPEG")
        print(f"이미지가 성공적으로 저장되었습니다: {save_path}")
 
url = "https://talk.kakaocdn.net/dna/cgCWKI/bl6V8v6z9s5/rQjDUMvgkLE9Nfv9vbdUPj/i_c75fc38092bd.png?credential=zf3biCPbmWRjbqf40YGePFLewdou7TIK&expires=1814139966&signature=PqVn8TnouSuPJzXFqWzWZRxBZqI%3D"

download_save(url=url)

In [ ]:
from utils import MyUtils
myutils = MyUtils(yam_file_path='./data/settings.yaml')

OUT_FOLDER = "./files/out"
file_paths = myutils.getListOfFiles(OUT_FOLDER)
for idx, file_path in enumerate(file_paths):
    if '.ipynb_checkpoints' not in file_path:
        print(file_path)
            

In [2]:
##langchain ######################
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
##################################

file_path = "./data/sample_text_01.txt"
chunk_size = 500
chunk_overlap = 30

loader = TextLoader(file_path=file_path) # text 파일을 로딩함.

#separators = ["\n\n", "\n", " ", "", "..PAGE:"]
separators = ["..PAGE:", "\n\n", "\n"]
text_splitter = RecursiveCharacterTextSplitter(separators=separators,
                                               chunk_size=chunk_size, 
                                               chunk_overlap=chunk_overlap)

split_docs = loader.load_and_split(text_splitter=text_splitter)

In [3]:
import re

# split 문서를 docs 리스트에 담음.
docs:list=[]
# 정규 표현식 패턴 정의
# ..PAGE:숫자 패턴과 숫자.숫자 패턴 정의
patterns = [r"\.\.PAGE:\d+", r"\d+\.\d+"]
    
for i in range(len(split_docs)):
    # 전처리 수행 => pattern이 ..PAGE:1 제거
    # 모든 패턴을 찾아 제거
    page_content = split_docs[i].page_content
    for pattern in patterns:
        page_content = re.sub(pattern, "", page_content)
    
    if len(page_content) > 10:
        docs.append(page_content)

In [4]:
docs

['\n휴온스글로벌\n기능비교 자료\n2024. 04. 03\n\n1. 사업 개요\n1. 사업 개요\n고객사\n사업명\n사용자수\n구축기간\n휴온스글로벌\n문서중앙화 구축\n1000User\n계약후 4개월\n구축 개요\n- 기존 보안파일서버 고도화 \n적용 솔루션\n\n시스템 구성도(3000User)\n2. 시스템 구성방안\n2. 시스템 구성방안\n. 시스템 구성도\n내부망\nL2스위치 \n1G 이상\nA\nS\nDB 서버 2식\nHeartbeat\n분산파일시스템\nA\nA\nL4스위치\nSAN 스위치\nSAN\nDB 서버 1식\n네트웍드라이브 속도 이슈 \n발생시 DB서버 병렬확장가능 \nNAS or 분산파일시스템\n향후 데이터 저장소 증설 및 2차 백업을 \n고려하여 분산파일시스템 추천\nHeartbeat\nA\nS\nF/W\nInternet\n인터넷망\nDMZ \nF/W\nMDS\nDS\nMpower G/W\n중계서버\n(기존제품활용)\nA\nA\n예시',
 '\n2. 시스템 구성방안\n2. 시스템 구성방안\n. 조견표\nHIDM 에서 I-HUB에 BR담당도 넘기는지 확인 필요?\n프로세스 LV1\n프로세스 LV2\n프로세스 LV3\n시스템 구분\n설명\n필요기능\n구현가능여부(O/△/X)\nDS WORK - 문서중앙화 연계\nDS WORK 업무그룹방 정보수신 및 폴더 구성\nDSWORK에서 정한 권한/공개 범위 수신\nDSWORK\n문서중앙화\no 업무그룹방명과 동일하게 폴더명을 정의한다.\no 업무그룹방의 공개수준을 참고해 폴더권한범위를 정의한다. (가안 : 전사공개/부서공개/미공개 등 )\n폴더/권한 정보 동기화\nO\n폴더에 대한 사용자 접근권한 정의\nDSWORK 업무그룹방에 속한 사용자정보 수신\nDSWORK\n문서중앙화\no 같은 업무그룹방에 있으면 권한을 일괄 부여\no 프로젝트 외 멤버는 문서중앙화에서 수정권한이 없다.\no 동일 업무그룹 내 사용자별 파일 접근수준 정의 방안 필요\n개설자/관리자 조직정보가 바뀌어도 업무그룹방-파일 정보 

In [ ]:
split_docs

In [ ]:

split_docs[1].page_content

In [ ]:
import re

def remove_page_pattern(text):
    # 정규 표현식 패턴 정의
    pattern = r"\.\.PAGE:\d+"
    
    # 패턴을 찾아 제거
    cleaned_text = re.sub(pattern, "", text)
    
    return cleaned_text

# 예시 텍스트
example_text = "..PAGE:1..PAGE:2"

# 함수 호출
cleaned_text = remove_page_pattern(example_text)
print(len(cleaned_text))
print(cleaned_text)
